# update

The comments section has a description of the features associated with tags and some thoughts and solutions to the overfitting problem.  
Please translate it by yourself for the time being. I will summarize those features soon.I'm so sorry. I'm so busy that I have to use Chinese for the time being.

评论区有关于tags相关特征的说明，并且有一些关于过拟合问题的思考与解决办法。  
麻烦各位朋友先自行翻译一下，我不久后会整理那些特征。太抱歉了，太忙了就只能暂时先用中文啦。  

# introduction

val:0.788 lb:No time to submit

Thanks to ragnar for this great script https://www.kaggle.com/ragnar123/riiid-model-lgbm  
Based on the notebook mentioned above, I did the following:
- Fixed some features that could cause overfitting, such as the "time difference between the current timestamp and the last time you made a mistake" feature.The original authors used answered_correctly under the same bundle for the frequency features, but in the test data, the answered_correctly under the current bundle is not known.
- Fixed some statistical features made using "prior_question_elapsed_time" and "prior_question_had_explanation".These two features mean "when was the last bundle" and "Whether was the last bundle solved", but the original authors did not align them to the previous bundle.
- The user attributes and title attributes are described from more angles, such as "user-que", "user-part", "user-tags", "part", "tags" (e.g., the highest correct rate of the tag under a question)...
- The Gensim library is used to TSVD the Tags matrix(Direct use of SKLearn'S TSVD can cause memory leaks, while the GenSIM library can stream the TSVD decomposition model training in batches), indexing the highest numeric column in each row as the topic of tags.The number of topics tried 20, 50, 75.There was no significant improvement in the number of topics from 75 to 50.The number of topics 50 is 0.0005 higher than the number of topics 20, but in order to prevent overfitting, I finally chose 20 topics.  It is worth mentioning that after describing the tags and user-tags attributes, the tags can be improved to a certain extent (0.001). On this basis, TSVD can improve the tags to a certain extent (0.0015).

The following directions can be improved:
- Adding a description of a "user-lecture" can be considered in the following directions, such as how many lectures the user has seen; when the user was working on a particular lecture, the current time difference between the last time a lecture with the same tag was viewed; and how often the lecture was viewed.I think these characteristics will lead to a very big improvement

Unfortunately, due to the problem of laboratory work, I only participated in the competition for less than a week, so that I didn't even have time to move to the online.This plan after my correction, I think the probability has been high will not overfit, I hope to give you some inspiration!Once again, I wish you all good results.  
***If this notebook is helpful to you, I hope you can do drop an upvote,thank you!***

介绍：
很感谢这个作者的开源贡献：https://www.kaggle.com/ragnar123/riiid-model-lgbm  
我在这个作者的工作上做了一些改进，具体如下：  
- 修正了一些可能会造成过拟合的特征，例如“当前时间戳和上一次做错题目的时间差”特征。原作者在做频率特征时，使用到了同一个bundle下的answered_correctly，但测试数据中，当前bundle下的answered_correctly是不知道的。
- 修正了使用“prior_question_elapsed_time”和“prior_question_had_explanation”所做的一些统计特征。这两个特征的意义是“上一个bundle所用时间”和“上一个bundle是否被解答”，但是原作者并没有将他们对齐到上一个bundle。
- 从更多的角度来对用户属性和标题属性进行描述，如“user-que”、“user-part”、“user-tags”、“part”、“tags”(例如在一个问题下正确率最高的标签的正确率)……
- 使用gensim库来对tags矩阵进行tsvd分解(直接使用sklearn的tsvd会造成内存泄漏，而gensim库可以流式的分批次进行tsvd分解模型的训练)，取每一行数值最高的那一列的索引来作为tags的主题。主题数尝试过20、50、75。主题数75较主题数50相比没有显著提升。主题数50较主题数20有0.0005的提升，但是为了防止过拟合，我最终选择了20个主题数。
- 值得一提的是，对tags以及user-tags的属性进行描绘后能得到一定提升（0.001），在此基础上，对tags进行tsvd又能够获得一定的提升（0.0015）

未来的一些可以改进的方向：
- 加入对“user-lecture”的描述，可以从以下方向进行考虑，例如用户看过多少个lecture、用户在做某个题时、上一次观看具有相同tag的讲座的距离当前的时间差、观看讲座的频率。我认为这些特征会带来非常大的提升

> ### 一些吐槽
很遗憾的是因为实验室工作的问题（大家真的不要像我一样把比赛的事告诉导师o(╥﹏╥)o，估计也没人像我这么傻），riiid我只玩了不到一星期，一直是在线下做（因为线上太慢了）。鸽了好久，回来把我线下分数对照线上排行榜一看已经惨不忍睹，昨晚想把这个迁到线上去来着，结果又被叫去搬砖，心态炸了，索性不搞了。   
这套方案经过我的修正后，我认为已经大概率不会过拟合了（也就是说估计如果能复现完，lb应该会比788要高），希望能给大家带来一些灵感！再次祝各位都能取得好成绩。（注：萌新第一次耍kaggle做开源，希望老哥们不要下次一定啦，给个赞吧(*^▽^*)，拜托了。有需要我实验具体数据的可以在评论区管我要，还有已经做好的tsvd分解表（可以直接merge到数据集上），还有已经做好的特征（存成了hdf5了）。小弟知无不言~）

# ***Note that this code cannot be run directly and only provides the feature generation method and a list of model parameters***

# Feature list

In [ ]:
['answered_correctly_u_avg',
 'answered_correctly_u_count',
 'answered_correctly_uq_count',
 'elapsed_time_u_avg_xiuzheng',
 'explanation_u_avg_xiuzheng',
 'part_correctly_q_mean',###线下
 'part_elapsed_time_mean',###线下
 'part_had_explanation_mean',###线下
 'part_user_count',
 'part_user_mean',
 'prior_question_elapsed_time',###原始
 'prior_question_had_explanation',###原始
 'question_correct_rate_last_20_mean',
 'question_correctly_q_count',###线下
 'question_correctly_q_mean',###线下
 'question_elapsed_time_mean',###线下
 'question_had_explanation_mean',###线下
 'tag_acc_count',###线下
 'tag_acc_max',###线下
 'tag_acc_min',###线下
 'tags_lsi',###线下
 'task_container_id',###原始
 'timestamp',###原始
 'timestamp_u_correct_recency_1',
 'timestamp_u_diff_1_2',
 'timestamp_u_diff_2_3',
 'timestamp_u_diff_3_end',
 'timestamp_u_incorrect_recency_1',
 'user_tag_acc_count',
 'user_tag_acc_max',
 'user_tag_acc_min']

# Features you can do offline (upload to notebook, then merge)

In [ ]:
### question_elapsed_time_mean and question_had_explanation_mean
temp=all_data.groupby("question_id")[["current_question_elapsed_time",'current_question_had_explanation']].mean()
temp.columns=['question_elapsed_time_mean','question_had_explanation_mean']
que_fea_group_1=temp
del temp
gc.collect()

###  question_correctly_q_count and question_correctly_q_mean
temp=all_data.groupby("question_id")['answered_correctly'].agg([("question_correctly_q_count","count"),("question_correctly_q_mean","mean")])
que_fea_group_2=temp
del temp
gc.collect()

### part_elapsed_time_mean、part_had_explanation_mean and part_correctly_q_mean
temp=all_data.groupby("part")[["current_question_elapsed_time",'current_question_had_explanation','answered_correctly']].mean()
temp.columns=['part_elapsed_time_mean','part_had_explanation_mean','part_correctly_q_mean']
part_fea_group=temp
del temp
gc.collect()



## Tags LSI

In [ ]:
### TSVD decomposition
from time import time 
import pandas as pd
import warnings
from time import time 
import warnings
from gensim import corpora,similarities,models
import pandas as pd

warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
time1=time()
from gensim import corpora,similarities,models
# test_data_all=pd.read_pickle("query_all.pickle")

all_data['tags'].fillna("-1",inplace=True)

class mycorpus(object):
    def __iter__(self):
        for index,doc in enumerate(all_data['tags']):
            if index%5000000==0:
                print(index)
            yield doc.split(' ')
corp = mycorpus()
# dictionary.add_documents(corp)
dictionary=corpora.Dictionary(corp)
time2=time()
print(time2-time1)



# test_data_all=pd.read_pickle("/home/kesci/work/query_all.pickle")

print(111523155656)
class MyCorpus(object):
    def __init__(self):
        print('MyCorpus')
    def __iter__(self):
        for index,doc in enumerate(all_data['tags']):
            if index%5000000==0:
                print(index)
            yield dictionary.doc2bow(doc.split(' '))
corpus = MyCorpus()
# corpus = [dictionary.doc2bow(text) for text in self.text]
tfidf_model = models.TfidfModel(corpus, id2word=dictionary)
# corpus_tfidf = tfidf_model[corpus]

tfidf_model.save("model/tfidf.model")
dictionary.save("model/dictionary.model")


from time import time 
import warnings
time1=time()
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
from gensim import corpora,similarities,models
import pandas as pd
# test_data_all=pd.read_pickle("/home/kesci/work/query_all.pickle")
corpus_tfidf = tfidf_model[corpus]
print(111523155656)
class MyCorpus(object):
    def __init__(self):
        print('MyCorpus')
    def __iter__(self):
        for index,doc in enumerate(corpus_tfidf):
            if index%1000000==0:
                print(index)
            yield doc
corpus3 = MyCorpus()
# corpus = [dictionary.doc2bow(text) for text in self.text]
lsi_model = models.LsiModel(corpus3, id2word=dictionary,chunksize=2500000,num_topics=50)
lsi_model.save("lsi_model/lsi_all_query.lsi")
time2=time()
print((time2-time1)/60)
# corpus_tfidf = tfidf_model[corpus]

def get_arg_max(single_list):
    max_index=0
    max_num=single_list[0][1]
    for index in range(len(single_list)-1):
        if max_num<single_list[index+1][1]:
            max_num=single_list[index+1][1]
            max_index=index+1
    return max_index
lsi_model=models.LsiModel.load("model/lsi_model/lsi_all_query_20.lsi")
dictionary=corpora.Dictionary.load("model/dictionary.model")
tfidf_model=corpora.Dictionary.load("model/tfidf.model")
all_data_lsi=[]
for text in tqdm(question['tags']):
    single_row_text=dictionary.doc2bow(text.split(' '))
    single_row_lsi_list=list(lsi_model[tfidf_model[single_row_text]])
    if len(single_row_lsi_list)==0:
        all_data_lsi.append(0)
        continue
    single_row_lsi=get_arg_max(single_row_lsi_list)
    all_data_lsi.append(single_row_lsi)
question['tags_lsi']=all_data_lsi
question.to_csv("temp/question_lsi20.csv",index=None)
question.head(50)

# Features that need to be constantly updated

In [ ]:
#  'answered_correctly_u_avg',
#  'answered_correctly_u_count',
#  'answered_correctly_uq_count',

#  'elapsed_time_u_avg_xiuzheng',
#  'explanation_u_avg_xiuzheng',

#  'part_user_count',
#  'part_user_mean',

#  'question_correct_rate_last_20_mean',

#  'timestamp_u_correct_recency_1',
#  'timestamp_u_incorrect_recency_1',

#  'timestamp_u_diff_1_2',
#  'timestamp_u_diff_2_3',
#  'timestamp_u_diff_3_end',

#  'user_tag_acc_count',
#  'user_tag_acc_max',
#  'user_tag_acc_min'
answered_correctly_u_count_dict = defaultdict(int)
answered_correctly_u_sum_dict = defaultdict(int)
answered_correctly_uq_dict = defaultdict(lambda: defaultdict(int))

elapsed_time_u_sum_dict = defaultdict(int)
explanation_u_sum_dict = defaultdict(int)
question_u_count_dict = defaultdict(int)
question_u_last_bundle_count_dict = defaultdict(int)

part_user_count_dict = defaultdict(lambda: defaultdict(int))
part_user_sum_dict = defaultdict(lambda: defaultdict(int))

question_correct_last_20_count_dict = defaultdict(int)
question_correct_last_20_sum_dict = defaultdict(int)
question_correct_last_20_all_dict = defaultdict(list)

timestamp_u_correct_dict = defaultdict(list)
timestamp_u_incorrect_dict = defaultdict(list)

timestamp_u_dict = defaultdict(list)

user_tag_acc_count_dict = defaultdict(lambda: defaultdict(int))
user_tag_acc_sum_dict = defaultdict(lambda: defaultdict(int))
# -----------------------------------------------------------------------
def add_features(update=True):
    # Client features
    answered_correctly_u_avg = np.zeros(len(all_data), dtype = np.float32)
    answered_correctly_u_count = np.zeros(len(all_data), dtype = np.float32)
    answered_correctly_uq_count = np.zeros(len(all_data), dtype = np.int32)

    elapsed_time_u_avg = np.zeros(len(all_data), dtype = np.float32)
    explanation_u_avg = np.zeros(len(all_data), dtype = np.float32)
    
    part_user_count = np.zeros(len(all_data), dtype = np.float32)
    part_user_mean = np.zeros(len(all_data), dtype = np.float32)
    
    question_correct_rate_last_20_sum = np.zeros(len(all_data), dtype = np.float32)
    
    timestamp_u_correct_recency_1 = np.zeros(len(all_data), dtype = np.float32)
    timestamp_u_incorrect_recency_1 = np.zeros(len(all_data), dtype = np.float32)
    
    timestamp_u_diff_1 = np.zeros(len(all_data), dtype = np.float32)
    timestamp_u_diff_2 = np.zeros(len(all_data), dtype = np.float32)
    timestamp_u_diff_3 = np.zeros(len(all_data), dtype = np.float32)
    
    user_tag_acc_count = np.zeros(len(all_data), dtype = np.float32)
    user_tag_acc_max = np.zeros(len(all_data), dtype = np.float32)
    user_tag_acc_min = np.zeros(len(all_data), dtype = np.float32)
    
    list_last_user_task_table=[]####定义数组 用来保存旧组的信息
    list_last_user_task_table_un_back=[]####定义数组 用来保存旧组的信息
#     for num, row in enumerate(tqdm(all_data[['user_id', 'answered_correctly', 'content_id', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'timestamp','task_container_id']].values)):
    flag_current_task=0
    all_data_temp=all_data[['user_id',"task_container_id", 'content_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'timestamp','part',"tags"]].values
    for num in tqdm(range(len(all_data))):
        row=all_data_temp[num]
        if num+1!=len(all_data):
            row2=all_data_temp[num+1]
        else:
            row2=[-100 for i in range(len(row))]
        
        
        ####*********  elapsed_time_u_avg_xiuzheng和explanation_u_avg_xiuzheng
        if row[6]!=0:##如果时间戳不是0的时候
            if flag_current_task==0:
                question_u_count_dict[row[0]]+=question_u_last_bundle_count_dict[row[0]]
                elapsed_time_u_sum_dict[row[0]]+=row[4]*question_u_last_bundle_count_dict[row[0]]
                explanation_u_sum_dict[row[0]]+=row[5]*question_u_last_bundle_count_dict[row[0]]
            elapsed_time_u_avg[num]= elapsed_time_u_sum_dict[row[0]]/question_u_count_dict[row[0]]
            explanation_u_avg[num] = explanation_u_sum_dict[row[0]]/question_u_count_dict[row[0]]
            ###⑥只需要当前组的prior（也就是上一组的平均时间或者是否解答），就可以计算了
        else:##时间戳为0的时候，肯定是不知道当前组的用时和解答情况的
            elapsed_time_u_avg[num]=np.nan
            explanation_u_avg[num] = np.nan
        flag_current_task=1
        
        ###①求这个特征，需要不断的记录上一组一共有多少道题，到最后用    （不断累加（每组多少道题*每道题平均时间））/总做题次数
        ###②需要把记录这组有多少道题放在后面计算，在前面计算平均时间并且填充到特征数组里
        list_last_user_task_table_un_back.append([row[0]])###没换人换组的时候，先不断保存旧组的信息,并且在换人换组的时候也要保存，以防那次信息没被用到
        if row[0]!=row2[0] or row[1]!=row2[1]:###换了一个task
            flag_current_task=0
            question_u_last_bundle_count_dict[row[0]]=len(list_last_user_task_table_un_back)
            list_last_user_task_table_un_back=[]###在即将换task的时候，把旧组需要换成新组（更换成新组之前，需要先把旧组的信息在上面用完）
            
        ####*********
        
        ####*********   answered_correctly_u_avg、answered_correctly_u_count和answered_correctly_uq_count
        if answered_correctly_u_count_dict[row[0]] != 0:
            answered_correctly_u_avg[num] = answered_correctly_u_sum_dict[row[0]] / answered_correctly_u_count_dict[row[0]]
            answered_correctly_u_count[num] = answered_correctly_u_count_dict[row[0]]
        else:
            answered_correctly_u_avg[num] = 0.67
            answered_correctly_u_count[num] = 0

        answered_correctly_uq_count[num] = answered_correctly_uq_dict[row[0]][row[2]]
        ####*********
        
        ####*********   part_user_count和part_user_mean
        if part_user_count_dict[row[0]][row[7]]==0:
            part_user_count[num] = 0
            part_user_mean[num] = 0.67
        else:
            part_user_count[num] = part_user_count_dict[row[0]][row[7]]
            part_user_mean[num] = part_user_sum_dict[row[0]][row[7]]/part_user_count_dict[row[0]][row[7]]
        ####*********
        
        ####*********   question_correct_rate_last_20_mean
#         question_correct_rate_last_20_sum[num]=question_correct_last_20_sum_dict[row[0]]
        ####*********
        
        
        ####*********   timestamp_u_correct_recency_1，timestamp_u_incorrect_recency_1
        if len(timestamp_u_correct_dict[row[0]]) == 0:
            timestamp_u_correct_recency_1[num] = np.nan
        elif len(timestamp_u_correct_dict[row[0]]) == 1:
            timestamp_u_correct_recency_1[num] = row[6] - timestamp_u_correct_dict[row[0]][0]
            
        if len(timestamp_u_incorrect_dict[row[0]]) == 0:
            timestamp_u_incorrect_recency_1[num] = np.nan
        elif len(timestamp_u_incorrect_dict[row[0]]) == 1:
            timestamp_u_incorrect_recency_1[num] = row[6] - timestamp_u_incorrect_dict[row[0]][0]
        ####*********

        ####*********   timestamp_u_diff_1_2，timestamp_u_diff_2_3，timestamp_u_diff_3_end
        if len(timestamp_u_dict[row[0]]) == 0:
            timestamp_u_diff_1[num] = np.nan
            timestamp_u_diff_2[num] = np.nan
            timestamp_u_diff_3[num] = np.nan
        elif len(timestamp_u_dict[row[0]]) == 1:
            timestamp_u_diff_1[num] = row[6] - timestamp_u_dict[row[0]][0]
            timestamp_u_diff_2[num] = np.nan
            timestamp_u_diff_3[num] = np.nan
        elif len(timestamp_u_dict[row[0]]) == 2:
            timestamp_u_diff_1[num] = row[6] - timestamp_u_dict[row[0]][1]
            timestamp_u_diff_2[num] = timestamp_u_dict[row[0]][1] - timestamp_u_dict[row[0]][0]
            timestamp_u_diff_3[num] = np.nan
        elif len(timestamp_u_dict[row[0]]) == 3:
            timestamp_u_diff_1[num] = row[6] - timestamp_u_dict[row[0]][2]
            timestamp_u_diff_2[num] = timestamp_u_dict[row[0]][2] - timestamp_u_dict[row[0]][1]
            timestamp_u_diff_3[num] = timestamp_u_dict[row[0]][1] - timestamp_u_dict[row[0]][0]

        ####*********
        
        ####*********   user_tag_acc_count，user_tag_acc_max，user_tag_acc_min
        if pd.isnull(row[8]):
            user_tag_acc_count[num]=np.nan
            user_tag_acc_max[num] = np.nan
            user_tag_acc_min[num] = np.nan
            continue
        else:
            tag_list_un_back=row[8].split()
            row_all_tag_sum=0
            row_all_tag_count=0
            row_max_tag_mean=-1###尽量搞小
            row_min_tag_mean=1000###尽量搞大

            for single_tag in tag_list_un_back:
                ###先做需要更新的###
                single_tag_sum=user_tag_acc_sum_dict[row[0]][single_tag]
                single_tag_count=user_tag_acc_count_dict[row[0]][single_tag]
                row_all_tag_sum+=single_tag_sum
                row_all_tag_count+=single_tag_count
                if single_tag_count==0:
                    single_tag_mean=0.67
                else:
                    single_tag_mean=single_tag_sum/single_tag_count
                row_max_tag_mean=max(single_tag_mean,row_max_tag_mean)
                row_min_tag_mean=min(single_tag_mean,row_min_tag_mean)
            if row_all_tag_count==0:
                user_tag_acc_count[num]=0
                user_tag_acc_max[num] = 0.67
                user_tag_acc_min[num] = 0.67
            else:
                user_tag_acc_count[num]=row_all_tag_count
                user_tag_acc_max[num] = row_max_tag_mean
                user_tag_acc_min[num] = row_min_tag_mean
        ####*********
        

        


        if update:
            answered_correctly_u_count_dict[row[0]] += 1
            answered_correctly_u_sum_dict[row[0]] += row[3]
            answered_correctly_uq_dict[row[0]][row[2]] += 1
            part_user_count_dict[row[0]][row[7]] += 1
            part_user_sum_dict[row[0]][row[7]] += row[3]
#             if question_correct_last_20_count_dict[row[0]]+1<=20:
#                 question_correct_last_20_count_dict[row[0]]+=1
#                 question_correct_last_20_sum_dict[row[0]]+=row[3]
#                 question_correct_last_20_all_dict[row[0]].append(row[3])
#             else:
#                 question_correct_last_20_sum_dict[row[0]]+=row[3]
#                 question_correct_last_20_sum_dict[row[0]]-=question_correct_last_20_all_dict[row[0]][-1]
#                 question_correct_last_20_all_dict[row[0]].pop(0)
#                 question_correct_last_20_all_dict[row[0]].append(row[3])
            
            tag_list=row[8].split()
            for single_tag in tag_list:
                ######更新一下 user-tag
                user_tag_acc_count_dict[row[0]][single_tag] += 1
                user_tag_acc_sum_dict[row[0]][single_tag] += row[3]
            
            #'user_id',"task_container_id", 'content_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'timestamp','part'
            list_last_user_task_table.append([row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7]])###没换人换组的时候，先不断保存旧组的信息,并且在换人换组的时候也要保存，以防那次信息没被用到
            if row[0]!=row2[0] or row[1]!=row2[1]:###换了一个task
                
                if len(timestamp_u_dict[row[0]]) == 3:
                    timestamp_u_dict[row[0]].pop(0)
                    timestamp_u_dict[row[0]].append(row[6])
                else:
                    timestamp_u_dict[row[0]].append(row[6])
                
                ####由于bundle下面包含很多question，每个question都有一个correct，所以需要用列表存储“旧的一整个组”的correct 
                for single_row_last_user_task_table in list_last_user_task_table:
                    if single_row_last_user_task_table[3]==1:
                        if len(timestamp_u_correct_dict[row[0]]) == 1:###这里，就使用row[0]就行，因为list_last_user_task_que_timestamp里全都是当前user-task的信息，而非下一个user-task的信息
                            timestamp_u_correct_dict[row[0]].pop(0)
                            timestamp_u_correct_dict[row[0]].append(single_row_last_user_task_table[6])
                        else:
                            timestamp_u_correct_dict[row[0]].append(single_row_last_user_task_table[6])
                    else:
                        if len(timestamp_u_incorrect_dict[row[0]]) == 1:###这里，就使用row[0]就行，因为list_last_user_task_que_timestamp里全都是当前user-task的信息，而非下一个user-task的信息
                            timestamp_u_incorrect_dict[row[0]].pop(0)
                            timestamp_u_incorrect_dict[row[0]].append(single_row_last_user_task_table[6])
                        else:
                            timestamp_u_incorrect_dict[row[0]].append(single_row_last_user_task_table[6])
                list_last_user_task_table=[]###在即将换task的时候，把旧组需要换成新组（更换成新组之前，需要先把旧组的信息在上面用完）

#     all_data['answered_correctly_u_avg']=answered_correctly_u_avg
#     all_data['answered_correctly_u_count']=answered_correctly_u_count
#     all_data['answered_correctly_uq_count']=answered_correctly_uq_count
#     all_data['elapsed_time_u_avg_xiuzheng']=elapsed_time_u_avg
#     all_data['explanation_u_avg_xiuzheng']=explanation_u_avg
#     all_data['part_user_count']=part_user_count
#     all_data['part_user_mean']=part_user_mean
#     all_data['timestamp_u_correct_recency_1']=timestamp_u_correct_recency_1
#     all_data['timestamp_u_incorrect_recency_1']=timestamp_u_incorrect_recency_1
#     all_data['timestamp_u_diff_1_2']=timestamp_u_diff_1
#     all_data['timestamp_u_diff_2_3']=timestamp_u_diff_2
#     all_data['timestamp_u_diff_3_end']=timestamp_u_diff_3
#     all_data['part_user_count']=part_user_count
#     all_data['part_user_mean']=part_user_mean
#     all_data['user_tag_acc_count']=user_tag_acc_count
#     all_data['user_tag_acc_max']=user_tag_acc_max
#     all_data['user_tag_acc_min']=user_tag_acc_min
    
       
add_features()



# model
When num_leaves are increased and max_depth is controlled within a certain range, the training speed and the result accuracy can be improved under large data volume (it has been verified on this data set, the effect is improved by about 0.001, and the training time is shortened by 1/3).

In [ ]:
model = lgb.LGBMClassifier(num_leaves=300,
                        max_depth=15,
                        learning_rate=0.1,
                        subsample=0.8,
                        feature_fraction=0.8,
                        random_state=2020,
                        n_estimators=200
                        )
lgb_model = model.fit(train_X[fea_list], 
                    train_Y,
                    eval_names=['train', 'valid'],
                    eval_set=[(train_X[fea_list], train_Y), (valid_X[fea_list], valid_Y)],
                    verbose=10,
                    eval_metric='auc',
                    early_stopping_rounds=10,
                     categorical_feature=['tags_lsi'])###                    categorical_feature=cate_feat


In [ ]:
feature_importance = lgb_model.feature_importances_
feature_importance = pd.DataFrame({'Features': fea_list, 'Importance': feature_importance}).sort_values('Importance', ascending = False)

fig = plt.figure(figsize = (10, 10))
fig.suptitle('Feature Importance', fontsize = 20)
plt.tick_params(axis = 'x', labelsize = 12)
plt.tick_params(axis = 'y', labelsize = 12)
plt.xlabel('Importance', fontsize = 15)
plt.ylabel('Features', fontsize = 15)
sns.barplot(x = feature_importance['Importance'], y = feature_importance['Features'], orient = 'h')
plt.show()


![fea_importance](http://pic.downk.cc/item/5feae6fc3ffa7d37b359cf3b.jpg)